3. для пользователей находим любимые категории
4. фильтруем то что порекомендовать по тому чтобы они были в любимых категориях

In [1]:
from lightfm import LightFM
from scipy.sparse import coo_matrix
import pandas as pd
import ml_metrics as metrics

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


# validate

In [2]:
user_hist_df = pd.read_csv('../data/raw/movies_dataset_10 months.csv')
user_hist_df['ts'] = pd.to_datetime(user_hist_df['ts'])
user_hist_df['month'] = user_hist_df['ts'].dt.month

test_1 = user_hist_df[user_hist_df['month'] == 6]
test_2 = user_hist_df[user_hist_df['month'] == 7]
train = user_hist_df[~user_hist_df['month'].isin([6, 7])]

In [3]:
correct_1 = test_1.drop_duplicates(['user_id', 'movie_id']).groupby('user_id')['movie_id'].apply(list).to_dict()
correct_2 = test_2.drop_duplicates(['user_id', 'movie_id']).groupby('user_id')['movie_id'].apply(list).to_dict()


In [4]:
train['event'] = 1

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
train = train.drop_duplicates(['user_id', 'movie_id', 'event'])

In [6]:
model = LightFM(no_components=10)

train_df = train[['user_id', 'movie_id', 'event']]

In [7]:
user_encode = {u: i for i, u in enumerate(train['user_id'].unique())}
item_encode = {u: i for i, u in enumerate(train['movie_id'].unique())}

In [8]:
user_decode = {v: k for k, v in user_encode.items()}
item_decode = {v: k for k, v in item_encode.items()}

In [9]:
n_users, n_items = len(user_encode), len(item_encode)

n_users, n_items

(2055, 4732)

In [10]:
train['user_id'] = train['user_id'].apply(lambda x: user_encode[x])
train['movie_id'] = train['movie_id'].apply(lambda x: item_encode[x])

In [11]:
train_coo = coo_matrix(
    (train['event'], (train['user_id'], train['movie_id'])), shape=(n_users, n_items))

In [93]:
moveis_views = train['movie_id'].value_counts()

items_to_recom = moveis_views[moveis_views>20].index
print(len(items_to_recom))

2091


preapre movie features and user prefs

In [94]:
movies_database = pd.read_csv('../data/raw/movies.csv')
movies_to_recom = [item_decode[x] for x in items_to_recom]
movies_database = movies_database[movies_database['id'].isin(movies_to_recom)]
item_features = movies_database[['id', 'genres']]
item_features['id'] = item_features['id'].apply(lambda x: item_encode[x])

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [95]:
item_features['genres'] = item_features['genres'].str.split(',')

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
item_features = item_features.dropna()

In [97]:
movie_genres = {}

for i in item_features.iterrows():
    movie_genres[i[1][0]] = i[1][1]

In [98]:
users_to_predict = [user_encode[x] for x in list(test_1['user_id'])]
items_to_predict = set(items_to_recom.copy())

In [99]:
user_hist = train[train['episode_id'] == 0].groupby('user_id')['movie_id'].apply(set).to_dict()
#user_hist = train.groupby('user_id')['movie_id'].apply(set).to_dict()
#user_hist = {user_encode[k]: v for k, v in user_hist.items()}

In [100]:
from collections import Counter

In [101]:
user_pref = {}
for user in users_to_predict:
    #print(user)
    user_pref[user] = {}
    #print(user_hist[user])
    for film in user_hist[user]:
        if movie_genres.get(film):
            for genre in movie_genres[film]:
                if user_pref[user].get(genre):
                    user_pref[user][genre] += 1
                else:
                    user_pref[user][genre] = 1
                    
    user_pref[user] = [key for key, val in Counter(user_pref[user]).most_common(5)]                
    

In [107]:
items_for_user = {}

for user in user_pref:
    items_for_user[user] = []
    for item in items_to_predict:
        try:
            if len(set(movie_genres[item]).intersection(user_pref[user])) >= 1:
                items_for_user[user].append(item)
        except KeyError:
            #items_for_user[user].append(item)
            pass
        

In [103]:
model = LightFM(no_components=10)

In [104]:
model.fit(train_coo, epochs=20)

In [108]:
recoms = {}
num_to_recom = 5
for user in users_to_predict:
    #print(user)
    items_to_score = list(set(items_for_user[user]).difference(user_hist[user]))
    if not items_to_score:
        items_to_score = list(items_to_predict)
    #print(len(items_to_score))
    predict = model.predict(user, items_to_score, num_threads=-1)
    top_recoms_id = sorted(range(len(predict)),
                           key=lambda i: predict[i])[-num_to_recom:]
    top_recoms_id.reverse()
    recoms[user_decode[user]] = [item_decode[items_to_score[i]]
                                 for i in top_recoms_id]

metrics.mapk(list(recoms.values()), list(correct_1.values()), 5)

0.003367088607594937

In [106]:
recoms = {}
num_to_recom = 5
for user in users_to_predict:
    #print(user)
    items_to_score = list(set(items_to_predict).difference(user_hist[user]))
    #print(len(items_to_score))
    predict = model.predict(user, items_to_score, num_threads=-1)
    top_recoms_id = sorted(range(len(predict)),
                           key=lambda i: predict[i])[-num_to_recom:]
    top_recoms_id.reverse()
    recoms[user_decode[user]] = [item_decode[items_to_score[i]]
                                 for i in top_recoms_id]

metrics.mapk(list(recoms.values()), list(correct_1.values()), 5)

0.0062974683544303805

0.006379746835443039

In [83]:
users_to_predict = [user_encode[x] for x in list(test_2['user_id'])]

In [84]:
recoms = {}
num_to_recom = 5
for user in users_to_predict:
    #print(user)
    items_to_score = list(items_to_predict.difference(user_hist[user]))
    predict = model.predict(user, items_to_score, num_threads=-1)
    top_recoms_id = sorted(range(len(predict)),
                           key=lambda i: predict[i])[-num_to_recom:]
    top_recoms_id.reverse()
    recoms[user_decode[user]] = [item_decode[items_to_score[i]]
                                 for i in top_recoms_id]

In [85]:
metrics.mapk(list(recoms.values()), list(correct_1.values()), 5)

0.006761603375527427

In [160]:
metrics.mapk(list(recoms.values()), list(correct_1.values()), 5)

0.007573839662447258

on LB 0.00786

after tuning 0.00619

# add movie features

In [40]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

In [41]:
movies_database = pd.read_csv('../data/raw/movies.csv')
movies_database.head()

,id,year,title,description,genres,director,actors,writers,music,art,producer,imdb_id,tmdb_id,imdb_rating,tmdb_rating,available_now
0,1,2008,The Girl from Monaco,A brilliant and neurotic attorney goes to Mona...,"Comedy,Drama",Anne Fontaine,"Fabrice Luchini,Helene de Saint-Pere,Jeanne Ba...","Anne Fontaine,Benoît Graffin",Philippe Rombi,NaN,"Philippe Carcassonne,Bruno Pesary,Christine Ra...",tt1139800,15342.0,5.8,4.8,yes
1,2,2008,Every Jack has a Jill,Jack is encouraged to take the romantic Paris ...,"Comedy,Melodrama",Jennifer Devoldère,"Justin Lee Bartha,Billy Boyd,Maurice Bénichou,...",Jennifer Devoldère,NaN,"Hervé Gallet,Alix Deschamps","Bruno Chiche,Nicolas Duval Adassovsky,Maxime R...",tt1094668,32338.0,6.1,5.4,yes
2,3,2009,Dorian Gray,A vain London playboy offers his soul in excha...,"Thriller,Sci-Fi",Oliver Parker,"Colin Firth,Nathan Rosen,Caroline Goodall,John...","Toby Finlay,Oscar Wilde",Charlie Mole,"John Beard,Ruth Myers,Niamh Coulter","Barnaby Thompson,Paul Brett,Simon Fawcett",tt1235124,23488.0,6.3,5.9,yes
3,4,2008,How to Lose Friends & Alienate People,A British writer struggles to fit in at a high...,"Comedy,Drama",Robert B. Willow,"Simon Pegg,Jeff Bridges,Danny Huston,Jillian A...","Peter Straughan,Toby Young",David Arnold,"John Beard,Ray Chan,Anthony Gasparro","Elizabeth Karlsen,Laurie Borg,Stephen Woolley",tt0455538,13092.0,6.4,6.1,yes
4,5,2009,Cell 211,The story of two men on different sides of a p...,"Action,Thriller",Daniel Monzon,"Luis Tosar,Vicente Romero,Fernando Soto,Luis Z...","Daniel Monzon,F.P. Gandull,Jorge Guerricaechev...",Roque Baños,"Antón Laguna,Montse Sancho","Álvaro Augustín,Juan Gordon,Emma Lustres",tt1242422,33273.0,7.6,7.4,yes


In [42]:
movies_to_recom = [item_decode[x] for x in items_to_recom]

In [23]:
movies_database = movies_database[movies_database['id'].isin(movies_to_recom)]
item_features = movies_database[['id', 'year', 'genres', 'imdb_rating', 'tmdb_rating']]
item_features.shape

(578, 5)

In [24]:
def convert_year(val):
    if val < 2000:
        cat = 0
    elif 2000<=val < 2010:
        cat = 1
    else:
        cat = 2
    return cat

item_features['year'] = item_features['year'].apply(convert_year)

user_hist_df = pd.read_csv('../data/raw/movies_dataset_10 months.csv')
movies = user_hist_df[user_hist_df['episode_id'] == 0]['movie_id'].unique()

item_features['movie'] = item_features['id'].apply(lambda x: 1 if x in movies else 0 )

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [28]:
test = item_features['genres'].fillna('hz').str.split(',')

mlb = MultiLabelBinarizer()
res = pd.DataFrame(mlb.fit_transform(test),
                   columns=mlb.classes_,
                   index=test.index)

to_drop = ['Family','Musical', 'Quarantine Info', 'hz', 'War']
res = res.drop(to_drop, 1)

In [29]:
item_features = pd.concat([item_features, res], 1).drop('genres', 1)

In [30]:
item_features = item_features.fillna(item_features.mean())

In [33]:
item_features['id'] = item_features['id'].apply(lambda x: item_encode[x])

In [109]:
missed_movies = set(item_decode.keys()).difference(set(item_features['id'].values))

item_features = item_features.sort_values('id').reset_index().drop('index', 1)

In [123]:
ind = 4298
for _id in missed_movies:
    item_features.loc[ind] = [_id] + [-1]*21
    ind+=1
    
item_features = item_features.sort_values('id')

In [139]:
movie_features = item_features.drop('id', 1).values

In [140]:
movie_features.shape

(4732, 21)